In [1]:
#Dependencies and Setup
import pandas as pd

In [2]:
#File path
file = "purchase_data.csv"

#Read and open csv
df_data = pd.read_csv(file)
df_data

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,101,Final Critic,4.19


In [3]:
#Total number of players
players_df = df_data.groupby('SN')['SN'].nunique()
count = players_df.count()
players_tot_df = pd.DataFrame({"Total Players":[count]})
players_tot_df

,Total Players
0,576


In [4]:
#Number of unique items
unique_items = df_data.groupby("Item Name")["Item Name"].unique().count()

#Average purchase price
average_price = '${:,.2f}'.format(round(df_data["Price"].mean(), 2))

#Total number of purchases
purchases_count = len(df_data)

#Total Revenue
revenue_df = '${:,.2f}'.format(df_data["Price"].sum())

purchasing_totals_df = pd.DataFrame({"Unique Items Count":[unique_items],
                                    "Average Price":average_price,
                                    "Number of Purchases":purchases_count,
                                    "Total Revenue":revenue_df})
purchasing_totals_df

,Unique Items Count,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [5]:
#Count and Percentage of Males
Males_count = df_data.loc[df_data["Gender"] == "Male"]
Males_count = len(Males_count.groupby("SN")["SN"].unique())
Males_per = '{:,.2%}'.format((Males_count/len(players_df)))

#Count and Percentage of Females
Females_count = df_data.loc[df_data["Gender"] == "Female"]
Females_count = len(Females_count.groupby("SN")["SN"].unique())
Females_per = '{:,.2%}'.format((Females_count/len(players_df)))

#Count and Percentage of Other
Other_count = df_data.loc[df_data["Gender"] == "Other / Non-Disclosed"]
Other_count = len(Other_count.groupby("SN")["SN"].unique())
Other_per = '{:,.2%}'.format((Other_count/len(players_df)))


gender_demo_df = pd.DataFrame({"Total Count":[Males_count,Females_count,Other_count],
                              "Percentage of Players":[Males_per,Females_per,Other_per]},index=["Male","Female","Other/Non-Disclosed"])

gender_demo_df                              

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other/Non-Disclosed,11,1.91%


In [6]:
df_data

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,101,Final Critic,4.19


In [69]:
#Purchase count
pur_count = df_data.groupby("Gender")["Purchase ID"].count()

#Average purchase price
avg_price = round((df_data.groupby("Gender")["Price"].mean()),2)

#Total purchase value 
tot_val = pur_count * avg_price

gen_pur_df = pd.DataFrame({"Purchase Count":pur_count,
                          "Average Purchase Price":avg_price,
                          "Total Purchase Value":tot_val})

#Average number of items by gender * average price per item by gender!!!!
sums = df_data.groupby("SN")["Price"].sum()
df = pd.DataFrame(sums)
df = df.reset_index()
sums = sums.sum()/len(sums)
dedupe = df_data.drop_duplicates(subset='SN')
merged = dedupe[['SN','Gender']].merge(df,on='SN')
merged_df = pd.DataFrame(merged.groupby('Gender').mean())
pd.merge(gen_pur_df, merged_df, left_index=True, right_index=True)


,Purchase Count,Average Purchase Price,Total Purchase Value,Price
Gender,,,,
Female,113,3.20,361.60,4.468395
Male,652,3.02,1969.04,4.065372
Other / Non-Disclosed,15,3.35,50.25,4.562727


In [131]:
#Bins and group names for age ranges 
bins = [0,10,15,20,25,30,35,40, 45]
group_names = ["<10","10-14","15-19","20-24","25-29","30-34","35-39",">45"]

#Use .cut 
df_data["Ages"] = pd.cut(df_data["Age"], bins, labels=group_names)
#Sort new df by age ranges
age_df = df_data.set_index("Age").sort_index()

#Purchase count
price_count = age_df.groupby("Ages")["Price"].count()
price_count_df = pd.DataFrame(price_count).rename(columns={"Price":"Purchase Count"})

#Average purchase price
avg_pur_price = round((age_df.groupby("Ages")["Price"].mean()),2)
avg_pur_price_df = pd.DataFrame(avg_pur_price).rename(columns={"Price":"Average Purchase Price"})

#Total purchase value 
total_purchase = round((age_df.groupby("Ages")["Price"].sum()),2)
total_purchase_df = pd.DataFrame(total_purchase).rename(columns={"Price":"Total Purchase Value"})

#Average purchase total per person by Age group
sums = age_df.groupby("SN")["Price"].sum()
df = pd.DataFrame(sums)
df = df.reset_index()
sums = sums.sum()/len(sums)
dedupe = age_df.drop_duplicates(subset='SN')
merged = dedupe[['SN','Ages']].merge(df,on='SN')
merged_df = pd.DataFrame(round((merged.groupby('Ages').mean()),2)).rename(columns={"Price":"Average Total Purchase per Person"})

merger_1 = pd.merge(price_count_df, avg_pur_price_df, left_index=True, right_index=True)
merger_2 = pd.merge(merger_1, total_purchase_df, left_index=True, right_index=True)
merger_final = pd.merge(merger_2, merged_df, left_index=True, right_index=True)

merger_final


,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Ages,,,,
<10,32,3.40,108.96,4.54
10-14,54,2.90,156.60,3.82
15-19,200,3.11,621.56,4.14
20-24,325,3.02,981.64,4.23
25-29,77,2.88,221.42,3.75
30-34,52,2.99,155.71,4.21
35-39,33,3.40,112.35,4.32
>45,7,3.08,21.53,3.08


In [134]:
top_spenders = df_data.groupby("SN")
top_spenders_price = top_spenders['Price'].sum()
top_spenders_count = top_spenders['Price'].count()
top_spenders_avg = round(top_spenders_price / top_spenders_count, 2)

top_spenders_df = pd.DataFrame({'Purchase Count': top_spenders_count,
                                'Average Purchase Price': top_spenders_avg,
                                'Total Purchase Value': top_spenders_price}).sort_values(ascending=False,by="Total Purchase Value")
top_spenders_df.head(5)


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,3.79,18.96
Idastidru52,4,3.86,15.45
Chamjask73,3,4.61,13.83
Iral74,4,3.40,13.62
Iskadarya95,3,4.37,13.10


In [236]:
purchase_count = df_data.groupby(["Item ID", "Item Name"]).count()["Price"].rename("Purchase Count")
average_price = df_data.groupby(["Item ID", "Item Name"]).mean()["Price"].rename("Average Purchase Price")
total_value = df_data.groupby(["Item ID", "Item Name"]).sum()["Price"].rename("Total Purchase Value")

total_purchase_analysis = pd.DataFrame({"Purchase Count":purchase_count,
                                   "Item Price":average_price,
                                   "Total Purchase Value":total_value}).sort_values("Purchase Count", ascending=False)
total_purchase_analysis.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
145,Fiery Glass Crusader,9,4.58,41.22
108,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77
82,Nirvana,9,4.90,44.10
19,"Pursuit, Cudgel of Necromancy",8,1.02,8.16


In [240]:
total_purchase_analysis.sort_values("Total Purchase Value", ascending=False).head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
82,Nirvana,9,4.90,44.10
145,Fiery Glass Crusader,9,4.58,41.22
92,Final Critic,8,4.88,39.04
103,Singed Scalpel,8,4.35,34.80
